### I. Research Questions:
1. Which psychiatric diagnosis are associated with higher ICU readmission rates? -> Do patient with mood disorders (296.x), psychotic disorders (295.x) or substance use (303-305) have higher [30]-day readmission rates compared to those without?
2. Does discontinuation or poor psychotropic medications at discharge predict higher readmissions? -> Does the discontinuation of psychotropic medication upon discharge correlate with increased readmission risk?
3. Can we combine patient-level features to predict ICU readmission risk? -> which combination of demographics, ICU, severity scores and medication changes is most predictive of readmission?

In [7]:
import pandas as pd

# Load data
admissions = pd.read_csv('./data/hosp/admissions.csv')
patients = pd.read_csv('./data/hosp/patients.csv')
diagnoses = pd.read_csv('./data/hosp/diagnoses_icd.csv')
icustays = pd.read_csv('./data/icu/icustays.csv')
prescriptions = pd.read_csv('./data/hosp/prescriptions.csv')

print("Initial admissions:", admissions.shape[0], "Patients:", patients['subject_id'].nunique(), "Notes:", diagnoses.shape[0])

# adult only
patients_adult = patients[patients['anchor_age'] >= 18]
admissions = admissions[admissions['subject_id'].isin(patients_adult['subject_id'])].copy()
diagnoses = diagnoses[diagnoses['hadm_id'].isin(admissions['hadm_id'])].copy()

print("After excluding <18:", admissions.shape[0],
      "Patients:", admissions['subject_id'].nunique(),
      "Notes:", diagnoses.shape[0])

def psych_range_match(code_str: str, icd_version: int) -> bool:

    if icd_version == 9:
        try:
            major = int(code_str.split('.')[0])
            return 290 <= major <= 319
        except ValueError:
            return False
    elif icd_version == 10:
        if code_str.startswith('F') and code_str[1:3].isdigit():
            major = int(code_str[1:3])
            return 1 <= major <= 99
    return False

# Apply row-wise without creating extra columns
diagnoses['psych_flag'] = diagnoses.apply(
    lambda row: psych_range_match(str(row['icd_code']).strip(), row['icd_version']),
    axis=1
)

psych_admissions = diagnoses[diagnoses['psych_flag']]['hadm_id'].unique()
admissions = admissions[admissions['hadm_id'].isin(psych_admissions)].copy()
diagnoses = diagnoses[diagnoses['hadm_id'].isin(admissions['hadm_id'])].copy()
print("Psych admissions:", admissions.shape[0], "Patients:", admissions['subject_id'].nunique(), "Notes:", diagnoses.shape[0])

# Merge in patient demographics
admissions = admissions.merge(
    patients[['subject_id', 'gender', 'anchor_age']], 
    on='subject_id', how='left'
)

# Sort by patient and time
admissions = admissions.sort_values(['subject_id', 'admittime']).copy()

# Output frequency of admission types
print("\nAdmission types:")
print(admissions['admission_type'].value_counts())

# Output basic demographics
print("\nPatient demographics:")
print(admissions[['gender', 'anchor_age', 'race']].describe(include='all'))

# Save to CSV
admissions.to_csv('./data/psych_admissions.csv', index=False)
diagnoses.to_csv('./data/psych_diagnoses.csv', index=False)

Initial admissions: 431088 Patients: 299777 Notes: 4752265
After excluding <18: 431088 Patients: 180747 Notes: 4752265
Psych admissions: 107692 Patients: 54291 Notes: 1482728

Admission types:
admission_type
EW EMER.                       30431
EU OBSERVATION                 26073
OBSERVATION ADMIT              24008
URGENT                          9495
SURGICAL SAME DAY ADMISSION     6269
DIRECT OBSERVATION              5040
DIRECT EMER.                    3588
ELECTIVE                        1979
AMBULATORY OBSERVATION           809
Name: count, dtype: int64

Patient demographics:
        gender     anchor_age    race
count   107692  107692.000000  107692
unique       2            NaN      33
top          F            NaN   WHITE
freq     57490            NaN   70498
mean       NaN      54.633993     NaN
std        NaN      18.695511     NaN
min        NaN      18.000000     NaN
25%        NaN      41.000000     NaN
50%        NaN      55.000000     NaN
75%        NaN      68.000000 

In [8]:
# patients.csv contains unique subject_id and gender
patients = pd.read_csv("data/hosp/patients.csv")
psych_admissions = pd.read_csv("data/psych_admissions.csv")

# Get total unique patients
total_any = patients["subject_id"].nunique()

# Get gender distribution for all patients
gender_any = patients["gender"].value_counts()

# For psych patients
psych_subject_ids = psych_admissions["subject_id"].unique()
psych_patients = patients[patients["subject_id"].isin(psych_subject_ids)]

total_psych = psych_patients["subject_id"].nunique()
gender_psych = psych_patients["gender"].value_counts()

# Calculate percentages
gender_any_pct = (gender_any / total_any * 100).round(2)
gender_psych_pct = (gender_psych / total_psych * 100).round(2)

# Print result
for gender in ['M', 'F']:
    print(f"{gender}: {gender_any[gender]} ({gender_any_pct[gender]}%) | Psych: {gender_psych[gender]} ({gender_psych_pct[gender]}%)")

M: 141227 (47.11%) | Psych: 24857 (45.78%)
F: 158550 (52.89%) | Psych: 29434 (54.22%)


In [9]:
# Step 3: Define ICU readmissions
print("ICU admissions:", icustays.shape[0], "Patients:", icustays['subject_id'].nunique())

icu_admissions = icustays.merge(admissions[['hadm_id', 'dischtime', 'admission_type', 'gender', 'anchor_age', 'race']], 
                                 on='hadm_id', how='inner')
icu_admissions = icu_admissions.sort_values(['subject_id', 'intime']).copy()
icu_admissions['next_admission'] = icu_admissions.groupby('subject_id')['intime'].shift(-1)

icu_admissions['dischtime'] = pd.to_datetime(icu_admissions['dischtime'])
icu_admissions['next_admission'] = pd.to_datetime(icu_admissions['next_admission'])

unique_psych_icu_patients = icu_admissions['subject_id'].nunique()
print("Psych patients with ≥1 ICU stay:", unique_psych_icu_patients)
total_psych_icu_stays = icu_admissions.shape[0]
print("Total ICU stays (psych):", total_psych_icu_stays)

# Flag 30-day ICU readmission
icu_admissions['readmitted_30d'] = (icu_admissions['next_admission'] - icu_admissions['dischtime']).dt.days <= 30
icu_admissions['readmitted_30d'] = icu_admissions['readmitted_30d'].fillna(False)

print("ICU Psych readmissions (30d):", icu_admissions['readmitted_30d'].sum(), 
      "Unique Patients:", icu_admissions[icu_admissions['readmitted_30d']]['subject_id'].nunique())
print(icu_admissions.head())
icu_admissions.to_csv('./data/icu_psy_readmissions.csv', index=False)

ICU admissions: 73141 Patients: 50934
Psych patients with ≥1 ICU stay: 14972
Total ICU stays (psych): 20441
ICU Psych readmissions (30d): 3025 Unique Patients: 2180
   subject_id   hadm_id   stay_id  \
0    10001725  25563031  31205490   
1    10001884  26184834  37510196   
2    10002013  23581541  39060235   
3    10002348  22725460  32610785   
4    10002618  22192064  35080100   

                                     first_careunit  \
0  Medical/Surgical Intensive Care Unit (MICU/SICU)   
1                Medical Intensive Care Unit (MICU)   
2      Cardiac Vascular Intensive Care Unit (CVICU)   
3                                Neuro Intermediate   
4                               Trauma SICU (TSICU)   

                                      last_careunit               intime  \
0  Medical/Surgical Intensive Care Unit (MICU/SICU)  2110-04-11 15:52:22   
1                Medical Intensive Care Unit (MICU)  2131-01-11 04:20:05   
2      Cardiac Vascular Intensive Care Unit (CVICU)  

In [10]:
# Step 4: Medication discontinuation
psych_meds = [
    # Antidepressants
    'amitriptyline', 'amoxapine', 'bupropion', 'citalopram', 'desipramine',
    'desvenlafaxine', 'doxepin', 'duloxetine', 'escitalopram', 'esketamine',
    'fluoxetine', 'imipramine', 'ketamine', 'levomilnacipran', 'mirtazapine',
    'nefazodone', 'nortriptyline', 'olanzapine/fluoxetine', 'paroxetine',
    'protriptyline', 'sertraline', 'trazodone', 'trimipramine',
    'venlafaxine', 'vilazodone', 'vortioxetine',

    # Antipsychotics
    'aripiprazole', 'aripiprazole lauroxil', 'asenapine', 'brexpiprazole',
    'cariprazine', 'chlorpromazine', 'clozapine', 'droperidol',
    'fluphenazine', 'haloperidol', 'iloperidone', 'loxapine',
    'lumateperone', 'lurasidone', 'olanzapine', 'olanzapine pamoate',
    'olanzapine/samidorphan', 'paliperidone', 'perphenazine',
    'pimavanserin', 'pimozide', 'quetiapine', 'risperidone',
    'thioridazine', 'thiothixene', 'trifluoperazine',
    'xanomeline/trospium chloride', 'ziprasidone',

    # Monoamine-oxidase inhibitors
    'isocarboxazid', 'phenelzine', 'selegiline', 'tranylcypromine',

    # Anxiolytics / sedatives / hypnotics
    'alprazolam', 'buspirone', 'chlordiazepoxide', 'clonazepam',
    'clorazepate', 'dexmedetomidine', 'diazepam', 'diphenhydramine',
    'eszopiclone', 'flurazepam', 'hydroxyzine', 'lorazepam', 'midazolam',
    'oxazepam', 'pentobarbital', 'ramelteon', 'suvorexant', 'tasimelteon',
    'temazepam', 'triazolam', 'zaleplon', 'zolpidem',

    # Substance-use treatments
    'acamprosate', 'buprenorphine', 'buprenorphine/naloxone',
    'disulfiram', 'naltrexone', 'topiramate',

    # Mood stabilizers
    'carbamazepine', 'divalproex sodium', 'lamotrigine', 'lithium',
    'oxcarbazepine', 'valproic acid',

    # Stimulants
    'amphetamine mixed salts', 'dexmethylphenidate',
    'dextroamphetamine', 'lisdexamfetamine', 'methamphetamine',
    'methylphenidate',

    # Miscellaneous psychotropics listed in the formulary
    'amantadine', 'atenolol', 'atomoxetine', 'clomipramine', 'clonidine',
    'fluvoxamine', 'gabapentin', 'guanfacine', 'metoprolol', 'nadolol',
    'pindolol', 'prazosin', 'pregabalin', 'propranolol', 'viloxazine'
]

# Total prescriptions before filtering
print("Total prescriptions:", prescriptions.shape[0], "Patients:", prescriptions['subject_id'].nunique())

# Filter prescriptions to adult psychiatric admissions
prescriptions = prescriptions[prescriptions['hadm_id'].isin(admissions['hadm_id'])].copy()
print("Prescriptions for psych admissions:", prescriptions.shape[0], "Patients:", prescriptions['subject_id'].nunique())
prescriptions.to_csv('./data/prescripted_patients.csv', index=False)

# Flag psychotropic medications
prescriptions['drug'] = prescriptions['drug'].astype(str).str.lower()
prescriptions['psych_med_flag'] = prescriptions['drug'].isin(psych_meds)
psych_prescriptions = prescriptions[prescriptions['psych_med_flag']].copy()
print("Psych prescriptions:", psych_prescriptions.shape[0], "Patients:", psych_prescriptions['subject_id'].nunique())
psych_prescriptions.to_csv('./data/psych_prescriptions.csv', index=False)

# Parse medication stop time
psych_prescriptions.loc[:, 'stoptime'] = pd.to_datetime(psych_prescriptions['stoptime'], errors='coerce')

# Determine last stop time per admission
last_prescriptions = psych_prescriptions.groupby('hadm_id')['stoptime'].max().reset_index()
last_prescriptions.rename(columns={'stoptime': 'last_psych_med_time'}, inplace=True)

# Only include admissions with psych meds
admissions_with_meds = admissions[admissions['hadm_id'].isin(last_prescriptions['hadm_id'])].copy()

# Merge last med stop time and calculate discontinuation
admissions_with_meds = admissions_with_meds.merge(last_prescriptions, on='hadm_id', how='left')
admissions_with_meds['med_discontinued'] = admissions_with_meds['last_psych_med_time'].isnull()

# Filter discontinued medication cases
discontinued_prescriptions = admissions_with_meds[admissions_with_meds['med_discontinued']].copy()

print("Psych Medication discontinued admissions:", discontinued_prescriptions.shape[0])
print("Unique patients with psych medication discontinued:", discontinued_prescriptions['subject_id'].nunique())

# Output enriched data
last_prescriptions.to_csv('./data/last_prescriptions_processed.csv', index=False)
discontinued_prescriptions.to_csv('./data/psych_meds_discontinued_with_demo.csv', index=False)

Total prescriptions: 15399811 Patients: 158422
Prescriptions for psych admissions: 4371500 Patients: 46372
Psych prescriptions: 316096 Patients: 39690
Psych Medication discontinued admissions: 12
Unique patients with psych medication discontinued: 12


In [11]:
import pandas as pd

chartevents = pd.read_csv("data/icu/chartevents.csv", usecols=["subject_id", "hadm_id", "charttime", "itemid", "valuenum"])
height_ids = [226730]
weight_ids = [226512]

height_df = chartevents[chartevents["itemid"].isin(height_ids)].dropna(subset=["valuenum"]).copy()
weight_df = chartevents[chartevents["itemid"].isin(weight_ids)].dropna(subset=["valuenum"]).copy()

height_first = height_df.sort_values(["subject_id", "hadm_id", "charttime"]).drop_duplicates(["hadm_id"])
weight_first = weight_df.sort_values(["subject_id", "hadm_id", "charttime"]).drop_duplicates(["hadm_id"])

height_first = height_first[["hadm_id", "valuenum"]].rename(columns={"valuenum": "height_cm"})
weight_first = weight_first[["hadm_id", "valuenum"]].rename(columns={"valuenum": "weight_kg"})

bmi_df = pd.merge(weight_first, height_first, on="hadm_id", how="inner")
bmi_df["height_m"] = bmi_df["height_cm"] / 100
bmi_df["bmi"] = bmi_df["weight_kg"] / (bmi_df["height_m"] ** 2)

bmi_df = bmi_df[["hadm_id", "bmi"]]
bmi_df.to_csv("data/derived/bmi.csv", index=False)
print("BMI data saved to data/derived/bmi.csv")


BMI data saved to data/derived/bmi.csv


In [12]:
import pandas as pd
import numpy as np

# Load required files
psych_admissions = pd.read_csv("./data/psych_admissions.csv")
charlson_df = pd.read_csv("./data/derived/charlson.csv")
sofa_df = pd.read_csv("./data/derived/first_day_sofa.csv")
sirs_df = pd.read_csv("./data/derived/sirs.csv")
admissions = pd.read_csv("./data/hosp/admissions.csv")
patients = pd.read_csv("./data/hosp/patients.csv")
icustays = pd.read_csv("./data/icu/icustays.csv")
prescriptions = pd.read_csv("./data/hosp/prescriptions.csv")
bmi_df = pd.read_csv("data/derived/bmi.csv")

# Preprocess and merge
psych_hadm_ids = set(psych_admissions['hadm_id'].unique())
demo = admissions.merge(patients, on='subject_id', how='left')
demo = demo[['subject_id', 'hadm_id', 'gender', 'anchor_age', 'race', 'admission_type']]
demo['is_psych'] = demo['hadm_id'].isin(psych_hadm_ids)
demo['is_readmitted'] = demo.duplicated(subset='subject_id', keep=False)
demo['has_icu_stay'] = demo['hadm_id'].isin(icustays['hadm_id'])

# Psych meds
psych_meds = ['haloperidol', 'risperidone', 'quetiapine', 'olanzapine', 'lorazepam', 'diazepam', 'alprazolam',
              'fluoxetine', 'sertraline', 'citalopram', 'escitalopram', 'paroxetine', 'venlafaxine', 'duloxetine',
              'bupropion', 'mirtazapine', 'valproate', 'lithium']
prescriptions['drug'] = prescriptions['drug'].astype(str).str.lower()
prescriptions['is_psych_med'] = prescriptions['drug'].isin(psych_meds)
psych_meds_hadm_ids = set(prescriptions[prescriptions['is_psych_med']]['hadm_id'].unique())
demo['has_psych_med'] = demo['hadm_id'].isin(psych_meds_hadm_ids)

# Merge scores
demo = demo.merge(charlson_df[['hadm_id', 'charlson_comorbidity_index']], on='hadm_id', how='left')
demo = demo.merge(sofa_df[['hadm_id', 'sofa']], on='hadm_id', how='left')
demo = demo.merge(sirs_df[['hadm_id', 'sirs']], on='hadm_id', how='left')
demo = demo.merge(bmi_df[["hadm_id", "bmi"]], on="hadm_id", how="left")

bmi_any = demo[np.isfinite(demo["bmi"])]["bmi"].mean()
bmi_psych = demo[(np.isfinite(demo["bmi"])) & (demo["is_psych"])]["bmi"].mean()

# Age groups
bins = [18, 30, 45, 60, 75, 90, 120]
labels = ['18-29', '30-44', '45-59', '60-74', '75-89', '90+']
demo['age_group'] = pd.cut(demo['anchor_age'], bins=bins, labels=labels, right=False)

# Build summary table
rows = []

# Age groups
for group in labels:
    any_patients = demo[demo['age_group'] == group]['subject_id'].nunique()
    psych_patients = demo[(demo['age_group'] == group) & (demo['is_psych'])]['subject_id'].nunique()
    rows.append(('Age Group: ' + group, any_patients, psych_patients))

# Gender
for gender in ['M', 'F']:
    any_patients = demo[demo['gender'] == gender]['subject_id'].nunique()
    psych_patients = demo[(demo['gender'] == gender) & (demo['is_psych'])]['subject_id'].nunique()
    rows.append(('Gender: ' + gender, any_patients, psych_patients))

# Admission type
for typ in demo['admission_type'].dropna().unique():
    any_patients = demo[demo['admission_type'] == typ]['subject_id'].nunique()
    psych_patients = demo[(demo['admission_type'] == typ) & (demo['is_psych'])]['subject_id'].nunique()
    rows.append(('Admission Type: ' + typ, any_patients, psych_patients))

# ICU stay
rows.append(('ICU Stay', demo[demo['has_icu_stay']]['subject_id'].nunique(),
             demo[(demo['has_icu_stay']) & (demo['is_psych'])]['subject_id'].nunique()))

# Readmission
rows.append(('Readmission', demo[demo['is_readmitted']]['subject_id'].nunique(),
             demo[(demo['is_readmitted']) & (demo['is_psych'])]['subject_id'].nunique()))

# Psych med
rows.append(('Psych Med', demo[demo['has_psych_med']]['subject_id'].nunique(),
             demo[(demo['has_psych_med']) & (demo['is_psych'])]['subject_id'].nunique()))

# Averages
rows.append(('Charlson Index (Avg)', demo['charlson_comorbidity_index'].mean(), demo[demo['is_psych']]['charlson_comorbidity_index'].mean()))
rows.append(('SOFA (Avg)', demo['sofa'].mean(), demo[demo['is_psych']]['sofa'].mean()))
rows.append(('SIRS (Avg)', demo['sirs'].mean(), demo[demo['is_psych']]['sirs'].mean()))

# BMI
rows.append(('BMI (Avg)', bmi_any, bmi_psych))


# Final table
final_df = pd.DataFrame(rows, columns=['Category', 'Any Patients', 'Psych Patients'])
final_df.to_csv('./data/summary_statistics.csv', index=False)